In [ ]:
# import libraries
import pandas as pd

import nltk

from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.corpus import stopwords

from nltk.tokenize import word_tokenize

from nltk.stem import WordNetLemmatizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string


In [ ]:
# download nltk corpus
import nltk

nltk.download('all')


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
# Load the amazon review dataset
# reviewText,Positive
df = pd.read_csv('/content/Restaurant_Reviews.csv')
df

,Review,Liked
0,Wow... Loved this place.,Yes
1,Crust is not good.,No
2,Not tasty and the texture was just nasty.,No
3,Stopped by during the late May bank holiday of...,Yes
4,The selection on the menu was great and so wer...,Yes
...,...,...
2215,The staff was untrained and unfamiliar with th...,No
2216,The portions were skimpy and left us hungry.,No
2217,The restaurant had unclean restrooms and unple...,No
2218,The waiters were slow and forgetful with our o...,No


In [ ]:
df['Review'] = df['Review'].str.lower()
df

,Review,Liked
0,wow... loved this place.,Yes
1,crust is not good.,No
2,not tasty and the texture was just nasty.,No
3,stopped by during the late may bank holiday of...,Yes
4,the selection on the menu was great and so wer...,Yes
...,...,...
2215,the staff was untrained and unfamiliar with th...,No
2216,the portions were skimpy and left us hungry.,No
2217,the restaurant had unclean restrooms and unple...,No
2218,the waiters were slow and forgetful with our o...,No


In [ ]:
import string
df['Review'] = df['Review'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
df['Review'].head(30)

0                                  wow loved this place
1                                     crust is not good
2              not tasty and the texture was just nasty
3     stopped by during the late may bank holiday of...
4     the selection on the menu was great and so wer...
5         now i am getting angry and i want my damn pho
6                    honeslty it didnt taste that fresh
7     the potatoes were like rubber and you could te...
8                              the fries were great too
9                                         a great touch
10                              service was very prompt
11                                    would not go back
12    the cashier had no care what so ever on what i...
13    i tried the cape cod ravoli chicken with cranb...
14    i was disgusted because i was pretty sure that...
15    i was shocked because no signs indicate cash only
16                                   highly recommended
17                waitress was a little slow in 

In [ ]:
df['Review'] = df['Review'].apply(lambda x: word_tokenize(x))
df['Review']

0                               [wow, loved, this, place]
1                                  [crust, is, not, good]
2       [not, tasty, and, the, texture, was, just, nasty]
3       [stopped, by, during, the, late, may, bank, ho...
4       [the, selection, on, the, menu, was, great, an...
                              ...                        
2215    [the, staff, was, untrained, and, unfamiliar, ...
2216    [the, portions, were, skimpy, and, left, us, h...
2217    [the, restaurant, had, unclean, restrooms, and...
2218    [the, waiters, were, slow, and, forgetful, wit...
2219    [the, food, was, poorly, cooked, and, tasted, ...
Name: Review, Length: 2220, dtype: object

In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
# Remove 'not' from the stopwords set
stop_words.discard('not')
df['Review'] = df['Review'].apply(lambda x: ' '.join([word for word in x if word not in (stop_words)]))
df['Review']

0                                         wow loved place
1                                          crust not good
2                                 not tasty texture nasty
3       stopped late may bank holiday rick steve recom...
4                             selection menu great prices
                              ...                        
2215                      staff untrained unfamiliar menu
2216                       portions skimpy left us hungry
2217        restaurant unclean restrooms unpleasant odors
2218                        waiters slow forgetful orders
2219                      food poorly cooked tasted burnt
Name: Review, Length: 2220, dtype: object

In [10]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download WordNet and the POS tagger if you haven't already
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

# Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# Function to convert POS tags from Treebank to WordNet tags
# treebank_tag is a POS tag
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return 'a'  # Adjective
    elif treebank_tag.startswith('V'):
        return 'v'  # Verb
    elif treebank_tag.startswith('N'):
        return 'n'  # Noun
    elif treebank_tag.startswith('R'):
        return 'r'  # Adverb
    else:
        return 'n'  # Default to noun if not recognized

# Function to lemmatize and apply POS tagging
def lemmatize_with_pos(text):
    tokens = word_tokenize(text) #perform tokenization
    pos_tags = nltk.pos_tag(tokens) #assigns POS tags to each word.
    lemmatized_words = [lemmatizer.lemmatize(word, get_wordnet_pos(pos_tag)) for word, pos_tag in pos_tags]
    return ' '.join(lemmatized_words)

# Applying lemmatization with POS tagging to 'Review' column in dataframe
df['Review'] = df['Review'].apply(lemmatize_with_pos)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [12]:
# Apply Feature Extraction (Bag of Words)
vectorizer = CountVectorizer(max_features=2000)
X = vectorizer.fit_transform(df['Review'])
y = df['Liked']
X

<2220x1958 sparse matrix of type '<class 'numpy.int64'>'
	with 11162 stored elements in Compressed Sparse Row format>

In [13]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
# Train a classifier
classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

In [15]:
# Make predictions
y_pred = classifier.predict(X_test)

In [16]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print("Accuracy:", accuracy)
print("Classification Report:\n", report)

Accuracy: 0.8918918918918919
Classification Report:
               precision    recall  f1-score   support

          No       0.89      0.91      0.90       236
         Yes       0.90      0.87      0.88       208

    accuracy                           0.89       444
   macro avg       0.89      0.89      0.89       444
weighted avg       0.89      0.89      0.89       444

